In [1]:
# Task 1: Markov Chain Sentence Generation
import random

class MarkovChain:
    def __init__(self):
        self.chain = {}

    def add_sentence(self, sentence):
        words = sentence.split()
        for i in range(len(words) - 1):
            if words[i] not in self.chain:
                self.chain[words[i]] = []
            self.chain[words[i]].append(words[i + 1])

    def generate_sentence(self, max_length=10):
        word = random.choice(list(self.chain.keys()))
        sentence = [word]
        for _ in range(max_length):
            if word not in self.chain:
                break
            next_word = random.choice(self.chain[word])
            sentence.append(next_word)
            word = next_word
        return ' '.join(sentence)


# Example usage
chain = MarkovChain()
chain.add_sentence("The cat is on the mat")
print(chain.generate_sentence())

is on the mat


In [2]:
# Task 2: Simple Autoencoder Model
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np

# Load dataset
(x_train, _), (x_test, _) = mnist.load_data()

# Preprocess data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train = x_train.reshape((len(x_train), -1))
x_test = x_test.reshape((len(x_test), -1))

# Define autoencoder model
input_dim = x_train.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile and train model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test, x_test))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.3851 - val_loss: 0.1898
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1795 - val_loss: 0.1532
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1494 - val_loss: 0.1324
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.1304 - val_loss: 0.1196
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.1188 - val_loss: 0.1108
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1108 - val_loss: 0.1050
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1053 - val_loss: 0.1009
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1015 - val_loss: 0.0980
Epoch 9/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0989 - val_loss: 0.0960
Epoch 10/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.0972 - val_loss: 0.0947
Epoch 11/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0955 - val_loss: 0.09

In [6]:
#  Use the Hugging Face transformers library to fine-tune a pre-trained GPT-2
# model on a custom text data and generate textI

!pip install transformers datasets

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load a small sample dataset (replace with your desired dataset)
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train[:1000]')

# Tokenize the dataset
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def tokenize_function(examples):
    return tokenizer(examples['text'])

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Adjust as needed (start with a small number)
    per_device_train_batch_size=4,  # Adjust based on your GPU memory
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=50,
    # The eval_dataset is now passed to the Trainer, not TrainingArguments
    evaluation_strategy="epoch",
)

# Define the model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Create and train the Trainer
# Pass eval_dataset to the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Pass the evaluation dataset here
)
trainer.train()

# Generate text
prompt = "This is a test prompt for"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=50, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 7


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: expected sequence of length 9 at dim 1 (got 209)

In [13]:
# Task 4: Simple RNN Text Generation
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Rest of the code remains the same.


# Sample custom dataset
data = """
Deep learning is a subset of machine learning.
Recurrent neural networks are a type of deep learning architecture.
They are particularly well-suited for sequence data.
"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Add 1 for padding token

# Prepare the sequences
sequences = []
for sentence in data.split('.'):
    tokens = tokenizer.texts_to_sequences([sentence.strip()])[0]
    for i in range(1, len(tokens)):
        sequences.append(tokens[:i + 1])

# Pad the sequences and prepare input and output
max_seq_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='pre')

X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=max_seq_length - 1),
    SimpleRNN(128, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Generate text
def generate_text(seed_text, next_words, model, tokenizer, max_seq_length):
    for _ in range(next_words):
        tokenized_sequence = tokenizer.texts_to_sequences([seed_text])[0]
        tokenized_sequence = pad_sequences([tokenized_sequence], maxlen=max_seq_length - 1, padding='pre')
        predicted_probs = model.predict(tokenized_sequence, verbose=0)
        predicted_word_index = np.argmax(predicted_probs)
        output_word = tokenizer.index_word.get(predicted_word_index, '')
        seed_text += ' ' + output_word
    return seed_text

# Test the model
seed_text = "Deep learning"
generated_text = generate_text(seed_text, next_words=10, model=model, tokenizer=tokenizer, max_seq_length=max_seq_length)
print("Generated Text:\n", generated_text)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0435 - loss: 3.0409
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.2174 - loss: 3.0263
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.3478 - loss: 3.0131
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4783 - loss: 3.0000
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6087 - loss: 2.9867
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.6957 - loss: 2.9727
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6957 - loss: 2.9575
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.6957 - loss: 2.9402
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7391 - loss: 2.9212
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7826 - loss: 2.8998
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7391 - loss: 2.8753
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7391 - los

In [14]:
#  Write a program to generate a sequence of text using an LSTM-based
# model in TensorFlow, trained on a custom data of sentences
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample custom dataset
data = """
Artificial Intelligence is a fascinating field of study.
Deep learning is a part of Artificial Intelligence.
LSTM networks are a type of Recurrent Neural Networks.
They excel at processing sequence data.
"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Include padding token

# Prepare sequences
sequences = []
for sentence in data.split('.'):
    tokens = tokenizer.texts_to_sequences([sentence.strip()])[0]
    for i in range(1, len(tokens)):
        sequences.append(tokens[:i + 1])

# Pad sequences and prepare input/output
max_seq_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='pre')

X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

# Build LSTM model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=max_seq_length - 1),
    LSTM(150, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Generate text
def generate_text(seed_text, next_words, model, tokenizer, max_seq_length):
    for _ in range(next_words):
        tokenized_sequence = tokenizer.texts_to_sequences([seed_text])[0]
        tokenized_sequence = pad_sequences([tokenized_sequence], maxlen=max_seq_length - 1, padding='pre')
        predicted_probs = model.predict(tokenized_sequence, verbose=0)
        predicted_word_index = np.argmax(predicted_probs)
        output_word = tokenizer.index_word.get(predicted_word_index, '')
        seed_text += ' ' + output_word
    return seed_text

# Test the model
seed_text = "Artificial Intelligence"
generated_text = generate_text(seed_text, next_words=10, model=model, tokenizer=tokenizer, max_seq_length=max_seq_length)
print("Generated Text:\n", generated_text)



Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0741 - loss: 3.1766
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.1111 - loss: 3.1698
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1111 - loss: 3.1626
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.1111 - loss: 3.1549
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1111 - loss: 3.1462
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.1111 - loss: 3.1361
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.1111 - loss: 3.1242
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.1111 - loss: 3.1098
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.1111 - loss: 3.0920
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1111 - loss: 3.0702
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.1111 - loss: 3.0435
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1111 -

In [15]:
#  Build a program that uses GPT-2 from Hugging Face to generate a story
# based on a custom prompt


from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load GPT-2 model and tokenizer
model_name = "gpt2"  # You can use 'gpt2-medium', 'gpt2-large', etc., for larger versions
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Function to generate a story
def generate_story(prompt, max_length=200, temperature=0.7, top_k=50, top_p=0.95):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate text using the model
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        num_return_sequences=1,
        do_sample=True
    )

    # Decode and return the generated text
    story = tokenizer.decode(output[0], skip_special_tokens=True)
    return story

# Custom prompt
prompt = "Once upon a time in a futuristic city,"

# Generate story
generated_story = generate_story(prompt, max_length=300)
print("Generated Story:\n", generated_story)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Story:
 Once upon a time in a futuristic city, you meet the greatest scientist in the galaxy. But when you meet a man who can control an entire universe, and to make such a leap, you must be ready to face him.


In [16]:
#  GRU-based Text Generation in Keras

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample text data
data = "Text generation using GRU is an interesting task in natural language processing."

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Prepare sequences
sequences = []
for i in range(1, len(data.split())):
    seq = tokenizer.texts_to_sequences([' '.join(data.split()[:i + 1])])[0]
    sequences.append(seq)

max_len = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

# Build the GRU-based model
model = Sequential([
    Embedding(vocab_size, 50, input_length=max_len - 1),
    GRU(128),
    Dense(vocab_size, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Text generation
def generate_text(prompt, next_words):
    for _ in range(next_words):
        tokenized = tokenizer.texts_to_sequences([prompt])[0]
        tokenized = pad_sequences([tokenized], maxlen=max_len - 1, padding='pre')
        prediction = np.argmax(model.predict(tokenized), axis=-1)
        output_word = tokenizer.index_word[prediction[0]]
        prompt += ' ' + output_word
    return prompt

print(generate_text("Text generation", 10))




Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0909 - loss: 2.5660
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.2727 - loss: 2.5554
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.2727 - loss: 2.5447
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.3636 - loss: 2.5338
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3636 - loss: 2.5224
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4545 - loss: 2.5104
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6364 - loss: 2.4975
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6364 - loss: 2.4836
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6364 - loss: 2.4682
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6364 - loss: 2.4512
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6364 - loss: 2.4320
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.6364 - lo

In [17]:
# GPT-2 Text Generation with Beam Search Decoding

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def generate_text_beam_search(prompt, num_beams=3, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    beam_output = model.generate(
        input_ids, max_length=max_length, num_beams=num_beams, early_stopping=True
    )
    return tokenizer.decode(beam_output[0], skip_special_tokens=True)

print(generate_text_beam_search("Once upon a time", num_beams=5))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, it was said, there would be a time when the world would be a better place.

It was a time when the world would be a better place.

It was a time when the world would be a


In [18]:
# . GPT-2 Text Generation with Custom Temperature

def generate_text_with_temperature(prompt, max_length=50, temperature=1.0):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids, max_length=max_length, temperature=temperature, do_sample=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_text_with_temperature("The universe is", temperature=0.7))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The universe is a collection of particles, which are called "space" particles. The universe is divided into four groups: cosmic, cosmic-space, cosmic-time, cosmic-time-space, and cosmic-time-time-space. We


In [21]:
# GPT-2 Text Generation with Temperature Sampling

for temp in [0.5, 1.0, 1.5]:
    print(f"Temperature: {temp}")
    print(generate_text_with_temperature("Creativity thrives", temperature=temp))
    print("-" * 50)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Temperature: 0.5


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Creativity thrives in the absence of an outside force.

The main reason for this is that the world is so much more complex and interconnected than it was before. While many people believe that the world is a complex place, there is one
--------------------------------------------------
Temperature: 1.0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Creativity thrives in people's minds. I know that I'm not alone with that in my ability to build relationships.

As an example: As a teen, I got into love with a boy who was very kind, loving, but
--------------------------------------------------
Temperature: 1.5
Creativity thrives in areas that need it most. People make choices not based on who needs a good quality and what exactly they'll spend in the long run, of course: they make decisions based on experience, not on money.

That
--------------------------------------------------


In [24]:
# . Custom Attention-Based Text Generation


import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample text data
data = """
Once upon a time, in a land far away, there was a magical kingdom.
The kingdom was filled with enchanted forests, mythical creatures, and brave heroes.
Every day, the people of the kingdom celebrated life and cherished their freedom.
"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Add 1 for padding token

# Prepare sequences
sequences = []
for i in range(1, len(data.split())):
    tokenized = tokenizer.texts_to_sequences([data.split()[:i + 1]])[0]
    for j in range(1, len(tokenized)):
        sequences.append(tokenized[:j + 1])

max_seq_length = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='pre')

X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

# Build the LSTM-based text generation model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=max_seq_length - 1),
    LSTM(128, return_sequences=False),  # Output one vector per sequence
    Dense(vocab_size, activation='softmax')  # Output probabilities for each word
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Generate text
def generate_text(seed_text, next_words, model, tokenizer, max_seq_length):
    for _ in range(next_words):
        tokenized = tokenizer.texts_to_sequences([seed_text])[0]
        tokenized = pad_sequences([tokenized], maxlen=max_seq_length - 1, padding='pre')
        prediction = np.argmax(model.predict(tokenized, verbose=0))
        output_word = tokenizer.index_word.get(prediction, '')
        seed_text += ' ' + output_word
    return seed_text

# Generate a story
seed_text = "Once upon a time"
generated_story = generate_text(seed_text, next_words=20, model=model, tokenizer=tokenizer, max_seq_length=max_seq_length)
print("Generated Story:\n", generated_story)


Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.1430 - loss: 3.3404
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.1873 - loss: 2.7049
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.2251 - loss: 2.2015
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.3476 - loss: 1.7497
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - accuracy: 0.6165 - loss: 1.4097
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.8158 - loss: 1.1283
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9186 - loss: 0.8835
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.9340 - loss: 0.6821
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.9490 - loss: 0.5209
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.9738 - loss: 0.4020
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.9786 - loss: 0.3118
Epoch 12/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/st